In [1]:
!pip install svgpath2mpl

In [2]:
from collections import namedtuple
import torch
from env.wildfire_gym import WildFireGym
from networks.lstm_ppo_net import LSTMPPONet
from torch.distributions import Categorical
import torch.nn.functional as F
import numpy as np

In [3]:
DT          = 0.5  # Time between wildfire updates            
DTI         = 0.1  # Time between aircraft decisions
n_actions = 2
height = width = 100
channels = 2
EPISODES_PER_BATCH = 1
TRAIN_FREQ  = 10
SAVE_FREQ = 20
GAMMA = 0.95

BATCH_SIZE = 128


LAMDA = 0.9
UPDATES = 10000
EPSILON = 0.1

EPOCHS = 10
NUM_PROCESSES = 1
BETA = 0.01
TAU = 1.0

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMPPONet(device,  channels, height, width, n_actions).to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)

In [5]:
Transition = namedtuple('Transition', ('belief_map', 'state_vector', 'action', 'reward', 'value', 'log_policy'))

class ReplayMemory:

  def __init__(self):
    self._memory = []
    
  def push(self, *args):
    """Save a transition"""
    self._memory.append(Transition(*args))

  @property
  def get_batch(self):
    return self._memory

  def __len__(self):
    return len(self._memory)

  def clear(self):
    self._memory.clear()

In [6]:
DT          = 0.5  # Time between wildfire updates            
DTI         = 0.1  # Time between aircraft decisions
fireEnv = ProbabilisticFireEnv(height, width)
dronesEnv = DronesEnv(height, width, DT, DTI) 
loss = None
i_episode = 1
SAVE_MODEL = 10
N_DRONES = 2
steps = 0
done = False

In [7]:
seed = fireEnv.reset()
dronesEnv.reset(seed)

memory = [ReplayMemory(), ReplayMemory()]
state_vectors = [None]*N_DRONES
hidden = [None]*N_DRONES
maps = [None]*N_DRONES
steps_for_episode = 0 

while True:
    
    for j in range(TRAIN_FREQ//int(2*DT/DTI)):

        observation = fireEnv.step()
        
        for d in range(N_DRONES):

            state_vectors[d] = dronesEnv.drones[d].state
            maps[d] = dronesEnv.drones[d].observation

            state_vectors[d] = torch.tensor(state_vectors[d], device=device, dtype=torch.float)
            maps[d] = torch.tensor(maps[d], device=device, dtype=torch.float)



        for i in range(int(DT/DTI)):
            
            steps += 1

            logits = [None]*N_DRONES
            values = [None]*N_DRONES
            actions = [None]*N_DRONES
            old_m = [None]*N_DRONES
            old_log_policy = [None]*N_DRONES
            rewards = [None]*N_DRONES

            for d in range(N_DRONES):

                logits[d], values[d], hidden[d] = model(maps[d], state_vectors[d], hidden[d])
                
                policy = F.softmax(logits[d], dim=1)
            
                old_m[d] = Categorical(policy)
    
                actions[d] = old_m[d].sample()
            
                old_log_policy[d] = old_m[d].log_prob(actions[d])
              
    
            rewards = dronesEnv.step([action.item() for action in actions], observation)
            
            
            for d in range(N_DRONES):
                memory[d].push(maps[d], state_vectors[d], actions[d], rewards[d], values[d].squeeze(), old_log_policy[d])
                state_vectors[d] = dronesEnv.drones[d].state
                maps[d] = dronesEnv.drones[d].observation

                state_vectors[d] = torch.tensor(state_vectors[d], device=device, dtype=torch.float)
                maps[d] = torch.tensor(maps[d], device=device, dtype=torch.float)

            if not fireEnv.fire_in_range(6):

                if i_episode % SAVE_MODEL == 0:
                    file_path = f'./lstm_ppo_weights.pt'
                    torch.save(model.state_dict(), file_path)
                    print('saved')

                for d in range(N_DRONES):
                    _, next_value, _ = model(maps[d], state_vectors[d], hidden[d])

                    next_value = next_value.squeeze()
                    
                    batch  = Transition(*zip(*memory[d].get_batch))


                    old_log_policies_batch = torch.cat(batch.log_policy).detach()
                

                    actions_batch = torch.cat(batch.action)


                    value_batch = torch.stack(batch.value).detach()
                

                    belief_map_batch = torch.cat(batch.belief_map)


                    state_vector_batch = torch.cat(batch.state_vector)


                    reward_batch = batch.reward 
                    
                    gae = 0
                    R = []

                    for value, reward in list(zip(value_batch, reward_batch))[::-1]:
                        gae = gae * GAMMA * TAU
                        gae = gae + reward + GAMMA * next_value.detach() - value.detach()
                        next_value = value
                        R.append(gae + value)

                    R = R[::-1]
                    R = torch.stack(R).detach()
                        
                    advantages = R - value_batch

                    indice = np.arange(0, len(memory[d]))
                    indices = np.split(indice, np.arange(BATCH_SIZE, len(memory[d]),BATCH_SIZE))
                    
                    for e_i in range(EPOCHS):
                        np.random.shuffle(indices)

                        for batch_indices in indices:
                            logits, value, _ = model(belief_map_batch[batch_indices], state_vector_batch[batch_indices], None, sequence_length=batch_indices.shape[0])
                            new_policy = F.softmax(logits, dim=1)
                            new_m = Categorical(new_policy)
                            new_log_policy = new_m.log_prob(actions_batch[batch_indices])
                            ratio = torch.exp(new_log_policy - old_log_policies_batch[batch_indices])
                            
                            actor_loss = -torch.mean(
                                torch.min(ratio * advantages[batch_indices],
                                    torch.clamp(ratio, 1.0 - EPSILON, 1.0 + EPSILON) *
                                    advantages[batch_indices]
                                )
                            )

                            critic_loss = F.smooth_l1_loss(R[batch_indices], value.squeeze())
                            entropy_loss = torch.mean(new_m.entropy())
                            total_loss = actor_loss + critic_loss - BETA * entropy_loss
                            optimizer.zero_grad()
                            total_loss.backward()
                            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                            optimizer.step()

                    print("Episode: {}. Total loss: {}".format(i_episode, total_loss))
                    memory[d].clear()
                    hidden[d] = None
                i_episode += 1
                seed = fireEnv.reset()
                dronesEnv.reset(seed)
                break

Episode: 1. Total loss: -0.38577336072921753
Episode: 1. Total loss: -0.27506178617477417
Episode: 2. Total loss: -0.022573454305529594
Episode: 2. Total loss: 0.030814310535788536
Episode: 3. Total loss: -0.093963123857975
Episode: 3. Total loss: 0.010571378283202648
Episode: 4. Total loss: 0.48500415682792664
Episode: 4. Total loss: -3.007322311401367
Episode: 5. Total loss: -3.2084686756134033
Episode: 5. Total loss: 1.6719194650650024
Episode: 6. Total loss: -1.0934234857559204
Episode: 6. Total loss: -9.371543884277344
Episode: 7. Total loss: 0.10459330677986145
Episode: 7. Total loss: -7.5029616355896
Episode: 8. Total loss: 1.010820746421814
Episode: 8. Total loss: 3.2413010597229004
Episode: 9. Total loss: -2.3888533115386963
Episode: 9. Total loss: -0.8025475740432739
saved
Episode: 10. Total loss: 3.961761236190796
Episode: 10. Total loss: 1.7014371156692505
Episode: 11. Total loss: 0.11677248030900955
Episode: 11. Total loss: 0.10249751806259155
Episode: 12. Total loss: 0.01